EA on OBB

In [2]:
from datetime import datetime, timedelta
import pandas as pd
import math

In [3]:
def import_data():
    gtfs_dir = '/Users/paulinaheine/Codes/BackupTravelPlanning/GTFS_OP_2024_obb-2/'

    # load data
    agency_df = pd.read_csv(gtfs_dir + 'agency.txt')
    stops_df = pd.read_csv(gtfs_dir + 'stops.txt')
    routes_df = pd.read_csv(gtfs_dir + 'routes.txt')
    trips_df = pd.read_csv(gtfs_dir + 'trips.txt')
    stop_times_df = pd.read_csv(gtfs_dir + 'stop_times.txt')
    calendar_df = pd.read_csv(gtfs_dir + 'calendar.txt')

    return agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df

agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df = import_data()

In [4]:
legcreation_df = stop_times_df.sort_values(['trip_id', 'stop_sequence']).reset_index(drop=True)
legcreation_df = legcreation_df.drop(columns=['stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled'])

In [5]:
legcreation_df = legcreation_df.merge(stops_df[['stop_id', 'stop_name']], on='stop_id', how='left')
legcreation_df = legcreation_df.drop(columns=['stop_id'])
print(legcreation_df)

                          trip_id arrival_time departure_time  stop_sequence  \
0              1.TA.1-1-j24-1.1.H     01:30:00       01:30:00              1   
1              1.TA.1-1-j24-1.1.H     01:34:00       01:35:00              2   
2              1.TA.1-1-j24-1.1.H     01:37:00       01:38:00              3   
3              1.TA.1-1-j24-1.1.H     01:41:00       01:41:00              4   
4              1.TA.1-1-j24-1.1.H     01:44:00       01:44:00              5   
...                           ...          ...            ...            ...   
315693  998.TA.20-SV9-G-j24-1.4.H     05:03:00       05:03:00              4   
315694  999.TA.20-SV9-G-j24-1.4.H     06:00:00       06:00:00              1   
315695  999.TA.20-SV9-G-j24-1.4.H     06:08:00       06:08:00              2   
315696  999.TA.20-SV9-G-j24-1.4.H     06:13:00       06:13:00              3   
315697  999.TA.20-SV9-G-j24-1.4.H     06:23:00       06:23:00              4   

                                   stop

In [6]:
#Shift the columns to get the arrival stop and time for each departure stop and time
legcreation_df['arrival_stop_name'] = legcreation_df['stop_name'].shift(-1)
legcreation_df['arrival_time'] = legcreation_df['arrival_time'].shift(-1)

print(legcreation_df)

                          trip_id arrival_time departure_time  stop_sequence  \
0              1.TA.1-1-j24-1.1.H     01:34:00       01:30:00              1   
1              1.TA.1-1-j24-1.1.H     01:37:00       01:35:00              2   
2              1.TA.1-1-j24-1.1.H     01:41:00       01:38:00              3   
3              1.TA.1-1-j24-1.1.H     01:44:00       01:41:00              4   
4              1.TA.1-1-j24-1.1.H     01:47:00       01:44:00              5   
...                           ...          ...            ...            ...   
315693  998.TA.20-SV9-G-j24-1.4.H     06:00:00       05:03:00              4   
315694  999.TA.20-SV9-G-j24-1.4.H     06:08:00       06:00:00              1   
315695  999.TA.20-SV9-G-j24-1.4.H     06:13:00       06:08:00              2   
315696  999.TA.20-SV9-G-j24-1.4.H     06:23:00       06:13:00              3   
315697  999.TA.20-SV9-G-j24-1.4.H         None       06:23:00              4   

                                   stop

In [7]:
# Filter out the last stop of each trip, as it has no next stop
legs_df = legcreation_df[legcreation_df['trip_id'] == legcreation_df['trip_id'].shift(-1)].copy()
#legs_df['arrival_stop_id']=legs_df['arrival_stop_id'].astype(int)

print(legs_df.head())

              trip_id arrival_time departure_time  stop_sequence  \
0  1.TA.1-1-j24-1.1.H     01:34:00       01:30:00              1   
1  1.TA.1-1-j24-1.1.H     01:37:00       01:35:00              2   
2  1.TA.1-1-j24-1.1.H     01:41:00       01:38:00              3   
3  1.TA.1-1-j24-1.1.H     01:44:00       01:41:00              4   
4  1.TA.1-1-j24-1.1.H     01:47:00       01:44:00              5   

                         stop_name                arrival_stop_name  
0                Bruck/Mur Bahnhof               Kapfenberg Bahnhof  
1               Kapfenberg Bahnhof        Kapfenberg Fachhochschule  
2        Kapfenberg Fachhochschule    St.Marein-St.Lorenzen Bahnhof  
3    St.Marein-St.Lorenzen Bahnhof  Allerheiligen-Mürzhofen Bahnhof  
4  Allerheiligen-Mürzhofen Bahnhof                 Kindberg Bahnhof  


In [8]:
legs_df = legcreation_df[legcreation_df['trip_id'] == legcreation_df['trip_id'].shift(-1)].copy()


print(legs_df.head())

              trip_id arrival_time departure_time  stop_sequence  \
0  1.TA.1-1-j24-1.1.H     01:34:00       01:30:00              1   
1  1.TA.1-1-j24-1.1.H     01:37:00       01:35:00              2   
2  1.TA.1-1-j24-1.1.H     01:41:00       01:38:00              3   
3  1.TA.1-1-j24-1.1.H     01:44:00       01:41:00              4   
4  1.TA.1-1-j24-1.1.H     01:47:00       01:44:00              5   

                         stop_name                arrival_stop_name  
0                Bruck/Mur Bahnhof               Kapfenberg Bahnhof  
1               Kapfenberg Bahnhof        Kapfenberg Fachhochschule  
2        Kapfenberg Fachhochschule    St.Marein-St.Lorenzen Bahnhof  
3    St.Marein-St.Lorenzen Bahnhof  Allerheiligen-Mürzhofen Bahnhof  
4  Allerheiligen-Mürzhofen Bahnhof                 Kindberg Bahnhof  


In [9]:
trips_df=trips_df.drop(columns=["service_id",'shape_id','trip_headsign','trip_short_name','direction_id','block_id'])
print(trips_df)

            route_id                   trip_id
0       10-A10-j24-1     1.TA.10-A10-j24-1.1.R
1       10-A10-j24-1     2.TA.10-A10-j24-1.2.R
2       10-A10-j24-1     3.TA.10-A10-j24-1.2.R
3       10-A11-j24-1  103.TA.10-A11-j24-1.24.H
4       10-A11-j24-1  104.TA.10-A11-j24-1.25.H
...              ...                       ...
34991  4-CX9-W-j24-1  95.TA.4-CX9-W-j24-1.50.R
34992  4-CX9-W-j24-1  96.TA.4-CX9-W-j24-1.50.R
34993  4-CX9-W-j24-1  97.TA.4-CX9-W-j24-1.50.R
34994  4-CX9-W-j24-1  98.TA.4-CX9-W-j24-1.51.R
34995  4-CX9-W-j24-1  99.TA.4-CX9-W-j24-1.52.R

[34996 rows x 2 columns]


In [10]:
def adjust_time_if_needed(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    if hours >= 24:
        adjusted_hours = hours - 24  # Reduce hours by 24
        return f"{adjusted_hours:02}:{minutes:02}:{seconds:02}"
    return time_str

In [11]:
legs_df['arrival_time'] = legs_df['arrival_time'].apply(adjust_time_if_needed)
legs_df['departure_time'] = legs_df['departure_time'].apply(adjust_time_if_needed)

In [12]:
legs_df = legs_df.merge(trips_df, on='trip_id', how='left')
print(legs_df.head)

<bound method NDFrame.head of                           trip_id arrival_time departure_time  stop_sequence  \
0              1.TA.1-1-j24-1.1.H     01:34:00       01:30:00              1   
1              1.TA.1-1-j24-1.1.H     01:37:00       01:35:00              2   
2              1.TA.1-1-j24-1.1.H     01:41:00       01:38:00              3   
3              1.TA.1-1-j24-1.1.H     01:44:00       01:41:00              4   
4              1.TA.1-1-j24-1.1.H     01:47:00       01:44:00              5   
...                           ...          ...            ...            ...   
280697  998.TA.20-SV9-G-j24-1.4.H     04:53:00       04:48:00              2   
280698  998.TA.20-SV9-G-j24-1.4.H     05:03:00       04:53:00              3   
280699  999.TA.20-SV9-G-j24-1.4.H     06:08:00       06:00:00              1   
280700  999.TA.20-SV9-G-j24-1.4.H     06:13:00       06:08:00              2   
280701  999.TA.20-SV9-G-j24-1.4.H     06:23:00       06:13:00              3   

         

In [13]:
#Creating legs tuples
legs_df['leg'] = list(zip(
    zip(legs_df['stop_name'], legs_df['departure_time']),
    zip(legs_df['arrival_stop_name'], legs_df['arrival_time']),
    legs_df['route_id']
))

legs_df = legs_df[['trip_id', 'leg']]
print(legs_df)

                          trip_id  \
0              1.TA.1-1-j24-1.1.H   
1              1.TA.1-1-j24-1.1.H   
2              1.TA.1-1-j24-1.1.H   
3              1.TA.1-1-j24-1.1.H   
4              1.TA.1-1-j24-1.1.H   
...                           ...   
280697  998.TA.20-SV9-G-j24-1.4.H   
280698  998.TA.20-SV9-G-j24-1.4.H   
280699  999.TA.20-SV9-G-j24-1.4.H   
280700  999.TA.20-SV9-G-j24-1.4.H   
280701  999.TA.20-SV9-G-j24-1.4.H   

                                                      leg  
0       ((Bruck/Mur Bahnhof, 01:30:00), (Kapfenberg Ba...  
1       ((Kapfenberg Bahnhof, 01:35:00), (Kapfenberg F...  
2       ((Kapfenberg Fachhochschule, 01:38:00), (St.Ma...  
3       ((St.Marein-St.Lorenzen Bahnhof, 01:41:00), (A...  
4       ((Allerheiligen-Mürzhofen Bahnhof, 01:44:00), ...  
...                                                   ...  
280697  ((Wien Lorenz-Böhler-Unfallkrankenhaus, 04:48:...  
280698  ((Wien Handelskai, 04:53:00), (Wien Floridsdor...  
280699  ((Wien 

Well.... i created legs.. but am quiet afraid of what will happen next

In [14]:
# Function to calculate probability of successful transfer between two subsequent legs
def calculate_transfer_probability(prev_leg: pd.Series, next_leg: pd.Series) -> float:
    if prev_leg.iloc[0] == next_leg.iloc[0]:
        return 1
    else:
        # >= as if transfer, cannot be arrival and departure at the same time
        if prev_leg.iloc[1][1][1] >= next_leg.iloc[1][0][1]:
            return 0
        #If tripId = next tripId , then probability = 1 since the traveler remains on the same line
        else:
            return 0.5
            # created random probability, just for the test
            #then have maybe exponential distribution ( less time- less probability, but if more that 15-30 it will stay like 0.8- 0.9)


In [15]:
#step by step
prev_leg = legs_df.iloc[1]
next_leg = legs_df.iloc[2]
a = calculate_transfer_probability(prev_leg,next_leg)
a

1

In [16]:
# i hope it is cumulative distribution
def calculate_cumulative_probability(itinerary) -> list[float]:
    cumulative_probabilities = [1] 
    for i in range(len(itinerary) - 1):
        prev_leg = itinerary[i]
        next_leg = itinerary[i+1]

        transfer_prob = calculate_transfer_probability(prev_leg, next_leg)
        cumulative_probabilities.append(transfer_prob)
    return cumulative_probabilities
  

In [17]:
def parse_custom_time_format(time_str: str) -> int:
    """
    Converts a time string like '25:06:30' into total minutes.
    """
    hours, minutes, seconds = map(int, time_str.split(":"))
    total_minutes = hours * 60 + minutes + seconds / 60
    return total_minutes

In [18]:
# if there is a chance to have all succesful transfers, is the arrival time in time budget
def calculate_arrival_probability(itinerary, start_time, time_budget) -> int:
    # Check if all previous transfers were successful
    if math.prod(calculate_cumulative_probability(itinerary)) > 0:  # Only proceed if all transfers are successful( naka can be possibly made)
        # Calculate the actual arrival time at the final leg
        destination_leg = itinerary[-1]
        destination_arrival_time = destination_leg.iloc[1][1][1]
        
        start_time = datetime.strptime(start_time, "%H:%M:%S")
        destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

        total_travel_time = destination_arrival_time - start_time

        # Check if total travel time is within the budget
        if total_travel_time <= time_budget:
            return 1  # Probability of 1 if arrival is within the time budget
        else:
            return 0  # Probability of 0 if arrival is beyond the time budget
    else:
        return 0  # Probability of 0 if any prior transfer was unsuccessful

In [19]:
def primary_itinerary_reliability(itinerary, start_time, time_budget) -> float:
    
    cumulative_probabilities = calculate_cumulative_probability(itinerary)
    product_of_probabilities= math.prod(cumulative_probabilities)
    arrival_probability = calculate_arrival_probability(itinerary,start_time,time_budget)

    reliability = arrival_probability * product_of_probabilities
    return reliability

In [20]:
#im not sure how the "backup" will look like (like is it starting from the origin or from the missed transfer point)
# backup = (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
#backup_legs = [list of backups excluding prim]
def backup_itinerary_reliability(itinerary, backup:tuple, start_time, time_budget) -> float:
    
    #change the format
    backup_itinerary = backup[1][:]
    backup_itinerary.insert(0,backup[0])
    arrival_probability = calculate_arrival_probability(backup_itinerary, start_time, time_budget)
    cumulative_probabilities = calculate_cumulative_probability(backup_itinerary)
    product_of_probabilities= math.prod(cumulative_probabilities)
    
    transfer_leg = backup[0] #leg of prim itn.
    transfer_point = transfer_leg.iloc[1][1][0]
    initial_transfer_prob = 1 # Default value in case no missed transfer is identified

    #but what if backup not to the last leg????
    for idx, leg in enumerate(itinerary[:-1]):  # Exclude last item since we're accessing i+1
        #basically comparing if it is the same leg
        if leg.iloc[1] is transfer_leg.iloc[1]:
            prev_leg = itinerary[idx]
            missed_leg = itinerary[idx + 1]
            initial_transfer_prob = calculate_transfer_probability(prev_leg,missed_leg)
            break
 
    backup_reliability = arrival_probability * product_of_probabilities * (1 - initial_transfer_prob)
    
    return backup_reliability


In [21]:
#Reliability of a complete itinerary 
#Backups will be set of backups of prim itineary
# backup itineary= (leg of the prim it where is transfer, [sequence of legs of backup starting from the next after transfer], reliability, arrival time)
def itinerary_reliability(itinerary : list[pd.Series],Backups : list[tuple], start_time:str, time_budget: timedelta) -> float:
    primary_reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
    if primary_reliability > 0 :
        added_reliability = 0
        for backup in Backups:
            #this will be fucked up, as we dont know yet how to make backups ( like on what format)
            backup_reliability = backup_itinerary_reliability(itinerary,backup,start_time,time_budget)
            added_reliability += backup_reliability
        added_reliability = min(added_reliability, 1-primary_reliability)
        complete_reliability = primary_reliability + added_reliability
        return complete_reliability
    else:
        return 0.0

lalal

In [22]:
def filter_network(start_time,time_budget):
    filtered_network= []
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    end_time = start_time + time_budget
    print(end_time)
    for index, row in legs_df.iterrows():
        leg_departure_time = datetime.strptime(row.iloc[1][0][1],"%H:%M:%S")
        leg_arrival_time = datetime.strptime(row.iloc[1][1][1],"%H:%M:%S")
        if leg_departure_time >= start_time and leg_departure_time<= end_time and leg_arrival_time <= end_time:
            filtered_network.append(row)
    print("Network",len(filtered_network))
    return filtered_network

In [23]:
#not sure if here we should add time dependancy
def search_adjecent_legs(node_id,arrival_time,start_time,time_budget,filtered_legs):
    arrival_time = datetime.strptime(arrival_time, "%H:%M:%S")
    adjecent_legs = []
    for i in range(0,len(filtered_legs)):
        leg = filtered_legs[i]
        leg_departure_time = datetime.strptime(leg.iloc[1][0][1],"%H:%M:%S")
        if leg.iloc[1][0][0] == node_id and leg_departure_time >= arrival_time :  # leg[0][0] corresponds to stop_id in the tuple (stop_id, departure_time)
            adjecent_legs.append(leg)
    print("ADJ",len(adjecent_legs))
    return adjecent_legs

In [24]:
def filter_adjecent_legs(adjecent_legs,shortest_path_trip_id):
    filtered_legs = {}
    for leg in adjecent_legs:
        destination_node = leg[1][1][0]
        trip_id = leg[0]
        if destination_node not in filtered_legs:
            filtered_legs[destination_node] = leg
        else:
            current_leg = filtered_legs[destination_node]
            # Compare and retain the shortest leg
            if datetime.strptime(leg[1][1][1], "%H:%M:%S") < datetime.strptime(current_leg[1][1][1], "%H:%M:%S"):
                filtered_legs[destination_node] = leg
            elif shortest_path_trip_id and trip_id == shortest_path_trip_id:
                filtered_legs[trip_id] = leg

    return list(filtered_legs.values())

In [25]:
# need to add to arrival probability
def travel_time(itinerary, start_time):
    destination_leg = itinerary[-1]
    destination_arrival_time = destination_leg.iloc[1][1][1]
        
    start_time = datetime.strptime(start_time, "%H:%M:%S")
    destination_arrival_time = datetime.strptime(destination_arrival_time, "%H:%M:%S")

    total_travel_time = destination_arrival_time - start_time
    return total_travel_time

In [26]:
def is_transfer(itinerary) -> bool:
    prev_leg = itinerary[-2]
    next_leg = itinerary[-1]
    if prev_leg.iloc[0] == next_leg.iloc[0]:
        return False
    else:
        return True

In [27]:
def find_min_index(LIST:list):
    """Find the index of the itinerary with the shortest duration."""
    #adjust for bachup
    min_index = 0
    min_value =  LIST[0][2]
    for i in range(1, len(LIST)):
        current_value = LIST[i][2]
        if current_value < min_value:
            min_value = current_value
            min_index = i

    return min_index

In [28]:
def update_visited_stops(visited_stops, leg):
    """Update the visited stops dictionary with a new leg."""
    destination_stop = leg.iloc[1][1][0]
    arrival_time = datetime.strptime(leg.iloc[1][1][1], "%H:%M:%S")
    trip_id = leg.iloc[0]
    visited_stops[destination_stop] = (arrival_time, trip_id)

In [29]:
def check_and_update_mrib(shortest_path, MRIB_reliability, MRIB, start_time, time_budget):
    """Check and update the most reliable path (MRIB) if applicable."""
    Backups = shortest_path[4][:]
    rel = itinerary_reliability(shortest_path[0], Backups, start_time, time_budget)
    if rel > MRIB_reliability:
        print("New most reliable path found")
        MRIB_reliability = rel
        MRIB = shortest_path
    return MRIB_reliability, MRIB

In [30]:
def can_visit_leg(leg, visited_stops, shortest_path_trip_id, destination_node):
    """
    Determines if a leg can be visited based on visited stops and time.

    Parameters:
        leg (list): The leg being evaluated.
        visited_stops (dict): Dictionary of visited stops with their earliest times.
        shortest_path_trip_id (str): Trip ID of the current path, if any.

    Returns:
        bool: True if the leg can be visited, False otherwise.
    """
    destination_stop = leg.iloc[1][1][0]  # Get destination stop ID
    arrival_time = datetime.strptime(leg.iloc[1][1][1], "%H:%M:%S")  # Arrival time
    trip_id = leg.iloc[0]  # Assuming trip_id is at index 0
    if destination_node != destination_stop:
        if destination_stop in visited_stops:
            visited_time, visited_trip_id = visited_stops[destination_stop]
            if trip_id == shortest_path_trip_id:
                return True
            else : 
                if arrival_time >= visited_time:
                    return False
                else:
                    visited_stops[destination_stop] = (arrival_time, trip_id)
                    print(leg)
                    return True

        else:
            visited_stops[destination_stop] = (arrival_time, trip_id)
            return True
    return True

In [31]:
def backup_search(shortest_path,shortest_next_itinerary,LIST_NEXT,destination_node,start_time,time_budget,filtered_legs,visited_stops):

    #Initial Setup
    MRB_reliability = 0.05
    MRB = None
    LIST_Backups = []
    visited_stops_b = visited_stops.copy()
    transfer_leg = shortest_path[0][-1]
    primary_itinerary = shortest_next_itinerary[0]
    for leg in primary_itinerary:
        update_visited_stops(visited_stops_b,leg)

    # no actually change so it that the departure of the next_itinerary -- if it is earlier, no point
    #transfer_arrival_time = datetime.strptime(transfer_leg.iloc[1][1][1], "%H:%M:%S")+timedelta(minutes=1)
    missed_leg_dep_time = datetime.strptime(primary_itinerary[-1].iloc[1][0][1], "%H:%M:%S") 

    #Transforming backups
    for i in range(0, len(LIST_NEXT)):
        #print("I",i) 
        #maube change later
        b_tail = LIST_NEXT[i][0][-1] # last leg- aka backup
        backup = (transfer_leg,[b_tail])
        #print("backup",backup)
        b_reliability = backup_itinerary_reliability(primary_itinerary,backup,start_time,time_budget)
        backup_full = (transfer_leg,[b_tail],b_reliability)
        b_duration = travel_time([b_tail],start_time)
        
        # so hear me out.. if we failed transfer, that means we are late, so we need the departure time of at least 30 sec after arrival?
        deparure_time = datetime.strptime(b_tail.iloc[1][0][1], "%H:%M:%S")
        if deparure_time > missed_leg_dep_time:
            if b_reliability > 0 and timedelta(seconds=0) < b_duration <= time_budget: # check initially so we dont waste time later
                LIST_Backups.append(backup_full)
                
            else:
                print("Backup Reliability 0 -1")
        else:
            print("No chance to get on the backup")
        update_visited_stops(visited_stops_b, b_tail)
            # Update visited stops
        
    #print("visited b stops", visited_stops_b)
    #print("list_backups",LIST_Backups)

    #Backup loop
    while len(LIST_Backups) > 0: 
        print("New backup iteration")  
        min_index_b = 0
        min_value_b =  LIST_Backups[0][1][-1].iloc[1][1][1]# the arrival time of the last leg of backup
        for i in range(1, len(LIST_Backups)):
            current_value_b = LIST_Backups[i][1][-1].iloc[1][1][1]
            if current_value_b < min_value_b:
                min_value_b = current_value_b
                min_index_b = i
    	
        shortest_backup = LIST_Backups.pop(min_index_b)
        print("Shortest backup",shortest_backup)
        
        b_tail = shortest_backup[1][-1]
        if b_tail.iloc[1][1][0] == destination_node:
            rel = backup_itinerary_reliability(shortest_next_itinerary[0],shortest_backup,start_time,time_budget)
            if rel > MRB_reliability:
                MRB_reliability = rel
                MRB = shortest_backup
                print("dest reached, MRB upd",MRB_reliability)
        else:
            
            next_legs_b = search_adjecent_legs(b_tail.iloc[1][1][0],b_tail.iloc[1][1][1],start_time,time_budget,filtered_legs)
            #next_legs_b_filtered = filter_adjecent_legs(next_legs_b,b_tail.iloc[0]) 
            
            next_legs_b = [
                leg for leg in next_legs_b 
                if can_visit_leg(leg, visited_stops_b, shortest_backup[1][-1].iloc[0],destination_node)
            ]
            
            next_legs_b = [
                leg for leg in next_legs_b 
                if can_visit_leg(leg, visited_stops_b, shortest_backup[1][-1].iloc[0],destination_node)
            ]


            #TRY to do smth with dublicates
            """b_filtered_legs = {}
            for leg in next_legs_b:   
                b_destination_node = leg.iloc[1][1][0]
                trip_id = leg.iloc[0]
                if b_destination_node not in b_filtered_legs:
                    b_filtered_legs[b_destination_node] = leg
                else:
                    current_leg = b_filtered_legs[b_destination_node]
                    # Compare and retain the shortest leg
                    if datetime.strptime(leg.iloc[1][1][1], "%H:%M:%S") < datetime.strptime(current_leg.iloc[1][1][1], "%H:%M:%S"):
                        b_filtered_legs[b_destination_node] = leg
                    elif trip_id == b_tail.iloc[0]:
                        b_filtered_legs[trip_id] = leg

            next_legs_b_filtered = list(b_filtered_legs.values())"""

            for leg in next_legs_b:
                backup_legs = shortest_backup[1][:]
                backup_legs.append(leg)
                backup = (transfer_leg,backup_legs)
                b_reliability = backup_itinerary_reliability(shortest_next_itinerary[0],backup,start_time,time_budget)
                backup_full = (transfer_leg,backup_legs,b_reliability)
                b_duration = travel_time(backup_legs,start_time)
                # maybe compare to MRIB rel
                if b_reliability > MRB_reliability and timedelta(seconds=0) < b_duration <= time_budget:
                    LIST_Backups.append(backup_full)
                   
                else:
                    print("Backup Reliability 0 -2")
                update_visited_stops(visited_stops_b, b_tail)
    # end of while len(LIST_Backups) > 0 loop       
    print("MRB", MRB)
    return MRB,MRB_reliability
                 
            
# end of while len(LIST_Backups) > 0 loop     

In [32]:
def find_path(origin_node: int,destination_node: int, start_time : str, time_budget: timedelta):
    
    '''Initial setup'''
    filtered_legs = filter_network(start_time,time_budget)
    MRIB_reliability = 0
    MRIB = None
    LISTofTRIPS = []
    visited_stops = {}
    n = 0

    visited_stops[origin_node] = (datetime.strptime(start_time, "%H:%M:%S"), "")


    ''' Search initial adjacent legs'''
    adjecent_legs = search_adjecent_legs(origin_node,start_time,start_time,time_budget,filtered_legs)
    for leg in adjecent_legs:
        itinerary  = [leg]
        reliability = primary_itinerary_reliability(itinerary,start_time,time_budget)
        duration = travel_time(itinerary,start_time)
        expected_arrival_time = leg.iloc[1][1][1]
        Backups = []
        if reliability > 0 and timedelta(seconds=0) < duration <= time_budget:
            LISTofTRIPS.append([itinerary,reliability,duration,expected_arrival_time,Backups])
        update_visited_stops(visited_stops, leg)
    #print("visited stops", visited_stops)
    
    '''Main Loop'''
    while LISTofTRIPS:
        n +=1 #just count
        print("New iteration", n)

        visited_stops_n = visited_stops.copy()

        # Find the shortest itinerary(no, min function is not working)
        min_index = find_min_index(LISTofTRIPS)
        shortest_path = LISTofTRIPS.pop(min_index)
        print("Shortest_path",shortest_path)
        tail = shortest_path[0][-1] # last leg of trip

        # Check if destination is reached
        if tail.iloc[1][1][0] == destination_node:
            MRIB_reliability, MRIB = check_and_update_mrib(shortest_path, MRIB_reliability, MRIB, start_time, time_budget)
            continue

        # Explore connections from the current tail
        #print("Search for the next connection")
        LIST_NEXT = []
        #set of all legs adjacent to tail leg m
        next_legs = search_adjecent_legs(tail.iloc[1][1][0],tail.iloc[1][1][1],start_time,time_budget,filtered_legs)
        #that was some idea, but bad idea
        #next_legs_filtered = filter_adjecent_legs(next_legs,tail.iloc[0])
        #another idea of keeping track of visited nodes
        print("before filter",len(next_legs))
        next_legs = [
            leg for leg in next_legs 
            if can_visit_leg(leg, visited_stops_n, shortest_path[0][-1].iloc[0],destination_node)
        ]
        print("1st filter",len(next_legs))
        next_legs = [
                leg for leg in next_legs 
                if can_visit_leg(leg, visited_stops_n,shortest_path[0][-1].iloc[0],destination_node)
            ]
        print("2nd filter",len(next_legs))
        
        for leg in next_legs:
            itinerary = shortest_path[0] + [leg] #combine previous legs and adjecent
            Backups = shortest_path[4][:] #transfer the backups
            reliability = itinerary_reliability(itinerary,Backups,start_time,time_budget)
            duration = travel_time(itinerary,start_time)
            expected_arrival_time = leg.iloc[1][1][1]

            # maybe compare to MRIB rel
            if reliability > 0 and timedelta(seconds=0) < duration <= time_budget: # check initially so we dont waste time later
                LIST_NEXT.append([itinerary,reliability,duration,expected_arrival_time,Backups])
            update_visited_stops(visited_stops_n, leg)
            
            #print("visited stops", visited_stops)
        print("list Next",LIST_NEXT)
        nn= 0 

        '''Sub Loop'''
        while LIST_NEXT:
            nn+=1
            print("New Subiteration", nn)

            min_index = find_min_index(LIST_NEXT)
            shortest_next_itinerary = LIST_NEXT.pop(min_index)
            print("shortest_next_itinerary",shortest_next_itinerary)

            if is_transfer(shortest_next_itinerary[0]) is False: 
                LISTofTRIPS.append(shortest_next_itinerary)
                print("direct_connecation, main list appended")
            else:

                '''Backup loop'''
                print('Transfer, search for backup')
                
                if len(LIST_NEXT) > 0 :
                    MRB, MRB_reliability = backup_search(shortest_path,shortest_next_itinerary,LIST_NEXT,destination_node,start_time,time_budget,filtered_legs,visited_stops)
                    if MRB != None:
                        shortest_next_itinerary[4] = shortest_next_itinerary[4][:]
                        shortest_next_itinerary[4].append(MRB)
                        print("APPENDEDBACKUP", shortest_next_itinerary)
                    else:
                        print("no reliable backup")
                    LISTofTRIPS.append(shortest_next_itinerary)
                else:
                    print("no backups")
                    


            print("no feasible connection, next new path")

             
    return MRIB_reliability, MRIB

In [33]:
pd.set_option('display.max_colwidth', None)

In [34]:
origin_node = "Klagenfurt Hauptbahnhof" # for now the departure node is 1
destination_node = "Villach Hauptbahnhof"
start_time = "14:30:00"
time_budget = timedelta(hours=2, minutes=0)

MRIB_reliability, MRIB = find_path(origin_node,destination_node,start_time,time_budget)

1900-01-01 16:30:00
Network 27682
ADJ 77
New iteration 1
Shortest_path [[trip_id                                                                   326.TA.1-S1-K-j24-1.120.H
leg        ((Klagenfurt Hauptbahnhof, 14:32:00), (Klagenfurt Lend Bahnhst, 14:34:00), 1-S1-K-j24-1)
Name: 148825, dtype: object], 1, datetime.timedelta(seconds=240), '14:34:00', []]
ADJ 52
before filter 52
trip_id                                                                   327.TA.1-S1-K-j24-1.120.H
leg        ((Klagenfurt Lend Bahnhst, 14:34:00), (Klagenfurt West Bahnhst, 14:36:00), 1-S1-K-j24-1)
Name: 149028, dtype: object
1st filter 3
2nd filter 1
list Next [[[trip_id                                                                   326.TA.1-S1-K-j24-1.120.H
leg        ((Klagenfurt Hauptbahnhof, 14:32:00), (Klagenfurt Lend Bahnhst, 14:34:00), 1-S1-K-j24-1)
Name: 148825, dtype: object, trip_id                                                                   326.TA.1-S1-K-j24-1.120.H
leg        ((Klagenfurt L

In [35]:
print(MRIB_reliability, MRIB)

1 [[trip_id                                                                   326.TA.1-S1-K-j24-1.120.H
leg        ((Klagenfurt Hauptbahnhof, 14:32:00), (Klagenfurt Lend Bahnhst, 14:34:00), 1-S1-K-j24-1)
Name: 148825, dtype: object, trip_id                                                                   326.TA.1-S1-K-j24-1.120.H
leg        ((Klagenfurt Lend Bahnhst, 14:34:00), (Klagenfurt West Bahnhst, 14:36:00), 1-S1-K-j24-1)
Name: 148826, dtype: object, trip_id                                                               326.TA.1-S1-K-j24-1.120.H
leg        ((Klagenfurt West Bahnhst, 14:36:00), (Krumpendorf Bahnhof, 14:39:00), 1-S1-K-j24-1)
Name: 148827, dtype: object, trip_id                                                          326.TA.1-S1-K-j24-1.120.H
leg        ((Krumpendorf Bahnhof, 14:40:00), (Pritschitz Bahnhst, 14:43:00), 1-S1-K-j24-1)
Name: 148828, dtype: object, trip_id                                                         326.TA.1-S1-K-j24-1.120.H
leg        ((Pri

In [67]:
MRIB[4][0][1][0][1]

C:\Users\Diana Lutska\AppData\Local\Temp\ipykernel_28924\3836976005.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  MRIB[4][0][1][0][1]


(('Klagenfurt Hauptbahnhof', '16:21:00'),
 ('Villach Hauptbahnhof', '16:44:00'),
 '10-A9-j24-1')